In [3]:
!pip install emoji wordcloud

In [2]:
import pandas as pd
from transformers import pipeline
import time
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import emoji

In [4]:
# Initialize sentiment analysis model
print("Loading sentiment analysis model...")
classifier = pipeline("sentiment-analysis")

# Read CSV files
print("\nReading CSV files...")
try:
    # Read the first CSV file
    file1_path = '/content/dataset_youtube-comments-scraper_2025-06-17_06-57-22-932.csv'
    df1 = pd.read_csv(file1_path)
    print(f"Successfully read {len(df1)} records from the first file")

    # Read the second CSV file
    file2_path = '/content/dataset_youtube-comments-scraper_2025-06-17_07-02-35-990.csv'  # Replace with your second file path
    df2 = pd.read_csv(file2_path)
    print(f"Successfully read {len(df2)} records from the second file")

except FileNotFoundError as e:
    print(f"Error: File not found - {e.filename}")
    print("Please ensure the CSV file exists in the specified directory")
    exit()

# Merge the two DataFrames
print("\nMerging the two datasets...")
merged_df = pd.concat([df1, df2], ignore_index=True)
print(f"Successfully merged {len(merged_df)} records")

# Check dataset overview
print(f"\nThe dataset contains {len(merged_df.columns)} columns")
print(f"Does 'comment' column exist: {'comment' in merged_df.columns}")

# Extract comment column and filter out null values
comments = merged_df['comment'].dropna().tolist()
print(f"\nFound {len(comments)} valid comments")

# Display first 3 example comments
print("\nExample Comments:")
for i, comment in enumerate(comments[:3]):
    print(f"{i+1}. {comment[:100]}...")

# Preprocess comments: replace emojis, remove URLs, etc.
def preprocess_comment(comment):
    # Replace emojis with text descriptions
    comment = emoji.demojize(comment)
    comment = comment.replace(":", "").replace("_", " ")

    # Remove URLs
    comment = re.sub(r'http\S+', '', comment)

    # Remove @usernames
    comment = re.sub(r'@\w+', '', comment)

    # Remove special characters
    comment = re.sub(r'[^\w\s]', '', comment)

    # Convert to lowercase
    comment = comment.lower()

    # Remove extra spaces
    comment = re.sub(r'\s+', ' ', comment).strip()

    return comment

print("\nPreprocessing comments...")
processed_comments = [preprocess_comment(comment) for comment in tqdm(comments)]

# Analyze sentiment
print("\nStarting sentiment analysis (this may take a few minutes)...")
results = []

# Use progress bar
for comment, proc_comment in tqdm(zip(comments, processed_comments), desc="Analysis Progress", total=len(comments)):
    try:
        # Truncate overly long text (model has length limit)
        truncated_text = proc_comment[:512] if len(proc_comment) > 512 else proc_comment

        # Analyze sentiment
        result = classifier(truncated_text)[0]
        results.append({
            'original_text': comment,
            'processed_text': proc_comment,
            'sentiment': result['label'],
            'confidence': result['score']
        })

    except Exception as e:
        # Handle possible errors
        print(f"Analysis error: {str(e)}")
        results.append({
            'original_text': comment,
            'processed_text': proc_comment,
            'sentiment': 'ERROR',
            'confidence': 0.0
        })

# Create results DataFrame
results_df = pd.DataFrame(results)

# Statistical analysis
print("\n=== Sentiment Analysis Statistics ===")
sentiment_counts = results_df['sentiment'].value_counts()
print(sentiment_counts)
print(f"\nPositive Proportion: {(sentiment_counts.get('POSITIVE', 0) / len(results_df) * 100):.1f}%")
print(f"Negative Proportion: {(sentiment_counts.get('NEGATIVE', 0) / len(results_df) * 100):.1f}%")

# Visualize sentiment distribution
def visualize_sentiment_distribution(sentiment_df):
    """Create pie chart and bar chart for sentiment analysis results"""
    # Sentiment count
    counts = sentiment_df['sentiment'].value_counts()

    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Pie chart: sentiment distribution
    ax1.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90,
            colors=['lightgreen', 'salmon', 'lightgray'])
    ax1.axis('equal')  # Equal aspect ratio ensures pie is circular
    ax1.set_title('Sentiment Distribution')

    # Bar chart: average confidence by sentiment
    confidence_by_sentiment = sentiment_df.groupby('sentiment')['confidence'].mean()
    ax2.bar(confidence_by_sentiment.index, confidence_by_sentiment.values,
            color=['lightgreen', 'salmon', 'lightgray'])
    ax2.set_ylim(0.8, 1.0)  # Set y-axis range to better show confidence differences
    ax2.set_title('Average Confidence by Sentiment')

    plt.tight_layout()
    plt.savefig('sentiment_visualization.png')
    plt.close()
    print("\nSentiment distribution visualization saved as 'sentiment_visualization.png'")

# Analyze keywords for positive and negative comments
def analyze_keywords(sentiment_df, top_n=10):
    """Analyze top keywords for positive and negative sentiments"""
    # English stopwords
    stopwords = set(['the', 'and', 'to', 'of', 'a', 'in', 'that', 'it', 'with', 'as', 'for', 'on', 'at', 'by', 'be', 'this', 'is', 'are', 'was', 'were', 'i', 'you', 'he', 'she', 'we', 'they', 'my', 'your', 'his', 'her', 'our', 'their'])

    # Positive comments
    positive_comments = sentiment_df[sentiment_df['sentiment'] == 'POSITIVE']['processed_text'].tolist()
    positive_words = []
    for comment in positive_comments:
        words = comment.split()
        positive_words.extend([word for word in words if word not in stopwords and len(word) > 2])
    positive_word_counts = Counter(positive_words)
    top_positive = positive_word_counts.most_common(top_n)

    # Negative comments
    negative_comments = sentiment_df[sentiment_df['sentiment'] == 'NEGATIVE']['processed_text'].tolist()
    negative_words = []
    for comment in negative_comments:
        words = comment.split()
        negative_words.extend([word for word in words if word not in stopwords and len(word) > 2])
    negative_word_counts = Counter(negative_words)
    top_negative = negative_word_counts.most_common(top_n)

    return top_positive, top_negative

# Generate word clouds
def generate_wordcloud(keywords, title, filename):
    """Generate word cloud from keyword list"""
    word_dict = dict(keywords)
    wordcloud = WordCloud(
        width=800, height=400, background_color='white', max_words=100
    ).generate_from_frequencies(word_dict)

    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=15)
    plt.tight_layout(pad=0)
    plt.savefig(filename)
    plt.close()
    print(f"Word cloud saved as '{filename}'")

# View high-confidence positive and negative samples
print("\n=== Top 5 High-Confidence Positive Samples ===")
positive_samples = results_df[results_df['sentiment'] == 'POSITIVE'].nlargest(5, 'confidence')
for idx, row in positive_samples.iterrows():
    print(f"Confidence: {row['confidence']:.2%}")
    print(f"Text: {row['original_text'][:150]}...")
    print("-" * 50)

print("\n=== Top 5 High-Confidence Negative Samples ===")
negative_samples = results_df[results_df['sentiment'] == 'NEGATIVE'].nlargest(5, 'confidence')
for idx, row in negative_samples.iterrows():
    print(f"Confidence: {row['confidence']:.2%}")
    print(f"Text: {row['original_text'][:150]}...")
    print("-" * 50)

# Save results
output_file = 'merged_youtube_sentiment_results.csv'
results_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"\nResults saved to: {output_file}")

# Optional: Merge sentiment results back into the original data
df_with_sentiment = merged_df.copy()
df_with_sentiment['sentiment'] = None
df_with_sentiment['sentiment_confidence'] = None

# Match and update sentiment results
for idx, row in df_with_sentiment.iterrows():
    if pd.notna(row['comment']):
        matching_result = results_df[results_df['original_text'] == row['comment']]
        if not matching_result.empty:
            df_with_sentiment.at[idx, 'sentiment'] = matching_result.iloc[0]['sentiment']
            df_with_sentiment.at[idx, 'sentiment_confidence'] = matching_result.iloc[0]['confidence']

# Save complete data
complete_output_file = 'merged_youtube_complete_with_sentiment.csv'
df_with_sentiment.to_csv(complete_output_file, index=False, encoding='utf-8')
print(f"Complete data with sentiment saved to: {complete_output_file}")

# Create visualizations
try:
    print("\nCreating visualizations...")
    visualize_sentiment_distribution(results_df)

    # Analyze keywords
    top_positive, top_negative = analyze_keywords(results_df)

    # Generate word clouds (requires wordcloud library)
    try:
        generate_wordcloud(top_positive, 'Positive Sentiment Keywords', 'positive_wordcloud.png')
        generate_wordcloud(top_negative, 'Negative Sentiment Keywords', 'negative_wordcloud.png')
    except ImportError:
        print("WordCloud library not installed. Skipping word cloud generation.")

except Exception as e:
    print(f"Visualization error: {str(e)}")

print("\nAnalysis Complete!")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Loading sentiment analysis model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0



Reading CSV files...
Successfully read 993 records from the first file
Successfully read 1459 records from the second file

Merging the two datasets...
Successfully merged 2452 records

The dataset contains 14 columns
Does 'comment' column exist: True

Found 2452 valid comments

Example Comments:
1. infold pls restuin q dgn ayel...
2. I keep coming back to this 😭...
3. "Put this on Spotify pleaseeeee" 
Me who uses Youtube Music cannot relate. :')...

Preprocessing comments...


100%|██████████| 2452/2452 [00:00<00:00, 3448.73it/s]



Starting sentiment analysis (this may take a few minutes)...


Analysis Progress: 100%|██████████| 2452/2452 [00:28<00:00, 86.74it/s]



=== Sentiment Analysis Statistics ===
sentiment
POSITIVE    1300
NEGATIVE    1152
Name: count, dtype: int64

Positive Proportion: 53.0%
Negative Proportion: 47.0%

=== Top 5 High-Confidence Positive Samples ===
Confidence: 99.99%
Text: This song made me have a good mood today 💖...
--------------------------------------------------
Confidence: 99.99%
Text: Beautiful... ❤ Happy spring time to everyone...
--------------------------------------------------
Confidence: 99.99%
Text: Yesss
I love this vibe! 💖...
--------------------------------------------------
Confidence: 99.99%
Text: This song is just so dreamy and fresh. Perfect for spring....
--------------------------------------------------
Confidence: 99.99%
Text: BEAUTIFULL! GLORIOUS! THIS IS JUST WHAT I NEEDED!!!...
--------------------------------------------------

=== Top 5 High-Confidence Negative Samples ===
Confidence: 99.98%
Text:  @Saskia666  Over time, this is no longer a mobile game, but more like a PC game forced to be p

In [4]:
# New: Word frequency statistics function
def analyze_word_frequency(sentiment_df, output_file='word_frequency.csv', top_n=100):
    """
    Analyze word frequency in all comments and save to CSV file

    Parameters:
        sentiment_df: DataFrame containing sentiment analysis results
        output_file: Path to save word frequency statistics
        top_n: Number of top frequent words to display
    """
    print("\n=== Starting word frequency analysis ===")

    # Load NLTK stopwords (use built-in list if NLTK not installed)
    try:
        import nltk
        from nltk.corpus import stopwords
        nltk.download('stopwords')
        stop_words = set(stopwords.words('english'))
        print("Successfully loaded NLTK English stopwords")
    except:
        # Use alternative stopword list
        stop_words = set(['the', 'and', 'to', 'of', 'a', 'in', 'that', 'it', 'with', 'as', 'for', 'on', 'at', 'by',
                         'be', 'this', 'is', 'are', 'was', 'were', 'i', 'you', 'he', 'she', 'we', 'they', 'my', 'your',
                         'his', 'her', 'our', 'their', 'what', 'which', 'who', 'whom', 'these', 'those', 'am', 'an'])
        print("NLTK not installed, using built-in English stopword list")

    # Extend stopword list with common meaningless words
    stop_words.update(['video', 'like', 'just', 'get', 'make', 'one', 'people', 'go', 'would',
                      'really', 'time', 'see', 'think', 'even', 'well', 'first', 'also', 'much'])

    # Count word frequency in all comments
    all_words = []
    for text in sentiment_df['processed_text']:
        if isinstance(text, str):
            words = text.lower().split()
            all_words.extend([word for word in words if word not in stop_words and len(word) > 2])

    # Calculate word frequencies
    word_counts = Counter(all_words)
    total_words = len(all_words)

    # Create word frequency DataFrame
    word_freq_df = pd.DataFrame({
        'Word': list(word_counts.keys()),
        'Frequency': list(word_counts.values())
    })

    # Calculate frequency percentage
    word_freq_df['Percentage'] = (word_freq_df['Frequency'] / total_words) * 100

    # Sort by frequency
    word_freq_df = word_freq_df.sort_values('Frequency', ascending=False).reset_index(drop=True)

    # Save to CSV
    word_freq_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Word frequency statistics saved to: {output_file}")

    # Print top_n frequent words
    print(f"\nTop {top_n} most frequent words:")
    print(word_freq_df.head(top_n))

    return word_freq_df

# New: Sentiment-based word frequency statistics
def analyze_sentiment_word_frequency(sentiment_df, output_prefix='sentiment_word_freq', top_n=50):
    """
    Analyze word frequency by sentiment and save to CSV files

    Parameters:
        sentiment_df: DataFrame containing sentiment analysis results
        output_prefix: Prefix for output file names
        top_n: Number of top frequent words to display per sentiment
    """
    print("\n=== Starting sentiment-based word frequency analysis ===")

    # Load NLTK stopwords (use built-in list if NLTK not installed)
    try:
        import nltk
        from nltk.corpus import stopwords
        nltk.download('stopwords')
        stop_words = set(stopwords.words('english'))
        print("Successfully loaded NLTK English stopwords")
    except:
        # Use alternative stopword list
        stop_words = set(['the', 'and', 'to', 'of', 'a', 'in', 'that', 'it', 'with', 'as', 'for', 'on', 'at', 'by',
                         'be', 'this', 'is', 'are', 'was', 'were', 'i', 'you', 'he', 'she', 'we', 'they', 'my', 'your',
                         'his', 'her', 'our', 'their', 'what', 'which', 'who', 'whom', 'these', 'those', 'am', 'an'])
        print("NLTK not installed, using built-in English stopword list")

    # Extend stopword list
    stop_words.update(['video', 'like', 'just', 'get', 'make', 'one', 'people', 'go', 'would',
                      'really', 'time', 'see', 'think', 'even', 'well', 'first', 'also', 'much'])

    # Split by sentiment
    positive_df = sentiment_df[sentiment_df['sentiment'] == 'POSITIVE']
    negative_df = sentiment_df[sentiment_df['sentiment'] == 'NEGATIVE']

    # Count word frequency in positive comments
    positive_words = []
    for text in positive_df['processed_text']:
        if isinstance(text, str):
            words = text.lower().split()
            positive_words.extend([word for word in words if word not in stop_words and len(word) > 2])

    # Count word frequency in negative comments
    negative_words = []
    for text in negative_df['processed_text']:
        if isinstance(text, str):
            words = text.lower().split()
            negative_words.extend([word for word in words if word not in stop_words and len(word) > 2])

    # Calculate word frequencies
    positive_counts = Counter(positive_words)
    negative_counts = Counter(negative_words)

    # Create word frequency DataFrames
    positive_freq_df = pd.DataFrame({
        'Word': list(positive_counts.keys()),
        'Frequency': list(positive_counts.values())
    })

    negative_freq_df = pd.DataFrame({
        'Word': list(negative_counts.keys()),
        'Frequency': list(negative_counts.values())
    })

    # Calculate frequency percentage
    total_positive = len(positive_words)
    total_negative = len(negative_words)

    positive_freq_df['Percentage'] = (positive_freq_df['Frequency'] / total_positive) * 100
    negative_freq_df['Percentage'] = (negative_freq_df['Frequency'] / total_negative) * 100

    # Sort by frequency
    positive_freq_df = positive_freq_df.sort_values('Frequency', ascending=False).reset_index(drop=True)
    negative_freq_df = negative_freq_df.sort_values('Frequency', ascending=False).reset_index(drop=True)

    # Save to CSV
    positive_freq_df.to_csv(f"{output_prefix}_positive.csv", index=False, encoding='utf-8')
    negative_freq_df.to_csv(f"{output_prefix}_negative.csv", index=False, encoding='utf-8')

    print(f"Positive sentiment word frequency saved to: {output_prefix}_positive.csv")
    print(f"Negative sentiment word frequency saved to: {output_prefix}_negative.csv")

    # Print top frequent words
    print(f"\nTop {top_n} most frequent words in positive comments:")
    print(positive_freq_df.head(top_n))

    print(f"\nTop {top_n} most frequent words in negative comments:")
    print(negative_freq_df.head(top_n))

    return positive_freq_df, negative_freq_df

# New: Word frequency visualization
def visualize_word_frequency(word_freq_df, title="Word Frequency Distribution", output_file="word_frequency.png", top_n=30):
    """
    Visualize word frequency distribution

    Parameters:
        word_freq_df: Word frequency DataFrame
        title: Chart title
        output_file: Path to save the visualization
        top_n: Number of top frequent words to display
    """
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns

        # Set font for Chinese characters (kept for compatibility, but will use available fonts)
        # plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC", "Arial"]

        # Select top_n most frequent words
        top_words = word_freq_df.head(top_n)

        # Create chart
        plt.figure(figsize=(12, 8))
        sns.barplot(x="Frequency", y="Word", data=top_words, palette="viridis")
        plt.title(title, fontsize=15)
        plt.xlabel("Frequency", fontsize=12)
        plt.ylabel("Word", fontsize=12)
        plt.tight_layout()

        # Save the chart
        plt.savefig(output_file, dpi=300, bbox_inches="tight")
        plt.close()

        print(f"Word frequency visualization saved to: {output_file}")

    except Exception as e:
        print(f"Word frequency visualization error: {str(e)}")

# Add word frequency analysis calls at the end of the original code
if __name__ == "__main__":
    # Perform word frequency statistics
    word_freq_df = analyze_word_frequency(results_df)

    # Sentiment-based word frequency statistics
    positive_freq_df, negative_freq_df = analyze_sentiment_word_frequency(results_df)

    # Word frequency visualization
    visualize_word_frequency(word_freq_df, "Overall Word Frequency Distribution", "word_frequency.png")
    visualize_word_frequency(positive_freq_df, "Positive Sentiment Word Frequency", "word_frequency_positive.png")
    visualize_word_frequency(negative_freq_df, "Negative Sentiment Word Frequency", "word_frequency_negative.png")


=== Starting word frequency analysis ===


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipython-input-4-965627367.py:177: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="Frequency", y="Word", data=top_words, palette="viridis")


Successfully loaded NLTK English stopwords
Word frequency statistics saved to: word_frequency.csv

Top 100 most frequent words:
        Word  Frequency  Percentage
0       face        621    3.346446
1     crying        502    2.705179
2   heartred        432    2.327962
3      heart        382    2.058522
4      zayne        272    1.465754
..       ...        ...         ...
95     caleb         26    0.140109
96    popper         26    0.140109
97      eyes         26    0.140109
98     swift         26    0.140109
99  thinking         26    0.140109

[100 rows x 3 columns]

=== Starting sentiment-based word frequency analysis ===
Successfully loaded NLTK English stopwords
Positive sentiment word frequency saved to: sentiment_word_freq_positive.csv
Negative sentiment word frequency saved to: sentiment_word_freq_negative.csv

Top 50 most frequent words in positive comments:
                Word  Frequency  Percentage
0           heartred        396    4.075332
1               face   

/tmp/ipython-input-4-965627367.py:177: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="Frequency", y="Word", data=top_words, palette="viridis")


Word frequency visualization saved to: word_frequency_positive.png


/tmp/ipython-input-4-965627367.py:177: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x="Frequency", y="Word", data=top_words, palette="viridis")


Word frequency visualization saved to: word_frequency_negative.png


基于LDA模型进行分析

In [1]:
!pip install gensim pyLDAvis

In [5]:
from gensim import corpora, models
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [6]:
def analyze_topics(sentiment_df, num_topics=5, passes=15):
    """
    使用LDA模型分析评论主题

    参数:
        sentiment_df: 情感分析结果DataFrame
        num_topics: 主题数量
        passes: 训练迭代次数
    """
    print("\n=== 开始主题模型分析 ===")

    # 文本预处理（适用于主题模型）
    all_texts = sentiment_df['processed_text'].tolist()

    # 过滤空文本
    valid_texts = [text for text in all_texts if text]

    # 创建词典
    dictionary = corpora.Dictionary(valid_texts)

    # 过滤低频词和高频词
    dictionary.filter_extremes(no_below=5, no_above=0.5)

    # 创建语料库
    corpus = [dictionary.doc2bow(text) for text in valid_texts]

    # 训练LDA模型
    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        passes=passes,
        random_state=42,
        alpha='auto',
        eta='auto'
    )

    # 评估模型
    coherence_model = CoherenceModel(model=lda_model, texts=valid_texts,
                                    dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    perplexity = lda_model.log_perplexity(corpus)

    print(f"LDA模型训练完成，主题数量: {num_topics}")
    print(f"模型困惑度: {perplexity:.4f}")
    print(f"模型一致性分数: {coherence:.4f}")

    # 保存主题关键词
    topic_keywords = {}
    for topic_id in range(num_topics):
        keywords = lda_model.show_topic(topic_id, topn=10)
        topic_keywords[topic_id + 1] = [word for word, _ in keywords]

    # 可视化主题
    try:
        vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
        pyLDAvis.save_html(vis_data, 'lda_visualization.html')
        print("主题可视化结果已保存为 'lda_visualization.html'")
    except Exception as e:
        print(f"主题可视化出错: {str(e)}")

    return lda_model, topic_keywords, dictionary, corpus

# 新增：时间序列趋势分析
def analyze_time_series(sentiment_df, time_column='publish_time', window_days=30):
    """
    分析情感随时间的变化趋势

    参数:
        sentiment_df: 带情感分析结果的DataFrame
        time_column: 时间列名（需为datetime类型）
        window_days: 时间窗口大小（天）
    """
    print("\n=== 开始时间序列分析 ===")

    # 确保时间列为datetime类型
    if not pd.api.types.is_datetime64_any_dtype(sentiment_df[time_column]):
        sentiment_df[time_column] = pd.to_datetime(sentiment_df[time_column])

    # 按时间排序
    sentiment_df = sentiment_df.sort_values(time_column)

    # 创建时间窗口（每月）
    sentiment_df['month'] = sentiment_df[time_column].dt.to_period('M')

    # 计算每月情感分布
    monthly_sentiment = sentiment_df.groupby(['month', 'sentiment'])['confidence'].count().unstack(fill_value=0)

    # 计算每月总评论数
    monthly_total = monthly_sentiment.sum(axis=1)

    # 计算情感比例
    monthly_proportion = monthly_sentiment.div(monthly_total, axis=0)

    # 可视化时间趋势
    plt.figure(figsize=(14, 7))

    # 绘制积极情感趋势
    if 'POSITIVE' in monthly_proportion.columns:
        plt.plot(monthly_proportion.index.astype(str), monthly_proportion['POSITIVE'],
                 marker='o', label='Positive', color='lightgreen')

    # 绘制消极情感趋势
    if 'NEGATIVE' in monthly_proportion.columns:
        plt.plot(monthly_proportion.index.astype(str), monthly_proportion['NEGATIVE'],
                 marker='o', label='Negative', color='salmon')

    plt.title('Sentiment Trend Over Time')
    plt.xlabel('Month')
    plt.ylabel('Proportion')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('sentiment_time_series.png')
    plt.close()

    print("情感时间趋势图已保存为 'sentiment_time_series.png'")

    return monthly_proportion

# 新增：情感-主题联合分析
def analyze_sentiment_topic(lda_model, sentiment_df, dictionary, corpus, top_n=5):
    """
    分析不同情感下的主题分布

    参数:
        lda_model: LDA模型
        sentiment_df: 情感分析结果
        dictionary: 词典
        corpus: 语料库
        top_n: 显示的主题数量
    """
    print("\n=== 开始情感-主题联合分析 ===")

    # 按情感分类
    positive_df = sentiment_df[sentiment_df['sentiment'] == 'POSITIVE']
    negative_df = sentiment_df[sentiment_df['sentiment'] == 'NEGATIVE']

    # 提取积极评论的语料库
    positive_indices = sentiment_df[sentiment_df['sentiment'] == 'POSITIVE'].index
    positive_corpus = [corpus[i] for i in positive_indices if i < len(corpus)]

    # 提取消极评论的语料库
    negative_indices = sentiment_df[sentiment_df['sentiment'] == 'NEGATIVE'].index
    negative_corpus = [corpus[i] for i in negative_indices if i < len(corpus)]

    # 计算积极评论的主题分布
    positive_topic_dist = np.zeros(lda_model.num_topics)
    for doc in positive_corpus:
        topics = lda_model.get_document_topics(doc)
        for topic, prob in topics:
            positive_topic_dist[topic] += prob

    # 计算消极评论的主题分布
    negative_topic_dist = np.zeros(lda_model.num_topics)
    for doc in negative_corpus:
        topics = lda_model.get_document_topics(doc)
        for topic, prob in topics:
            negative_topic_dist[topic] += prob

    # 归一化
    if len(positive_corpus) > 0:
        positive_topic_dist /= len(positive_corpus)

    if len(negative_corpus) > 0:
        negative_topic_dist /= len(negative_corpus)

    # 创建主题分布DataFrame
    topic_df = pd.DataFrame({
        'Topic': range(1, lda_model.num_topics + 1),
        'Positive_Proportion': positive_topic_dist,
        'Negative_Proportion': negative_topic_dist
    })

    # 可视化情感-主题分布
    plt.figure(figsize=(12, 7))

    x = topic_df['Topic']
    width = 0.35

    plt.bar(x - width/2, topic_df['Positive_Proportion'], width, label='Positive', color='lightgreen')
    plt.bar(x + width/2, topic_df['Negative_Proportion'], width, label='Negative', color='salmon')

    plt.title('Topic Distribution by Sentiment')
    plt.xlabel('Topic')
    plt.ylabel('Proportion')
    plt.xticks(x)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('sentiment_topic_distribution.png')
    plt.close()

    print("情感-主题分布图已保存为 'sentiment_topic_distribution.png'")

    # 输出各情感下的热门主题
    print("\n积极评论中的热门主题:")
    for i, topic in enumerate(topic_df.sort_values('Positive_Proportion', ascending=False).head(top_n).itertuples()):
        print(f"主题 {topic.Topic}: 比例 {topic.Positive_Proportion:.2%}")

    print("\n消极评论中的热门主题:")
    for i, topic in enumerate(topic_df.sort_values('Negative_Proportion', ascending=False).head(top_n).itertuples()):
        print(f"主题 {topic.Topic}: 比例 {topic.Negative_Proportion:.2%}")

    return topic_df

# 在原代码末尾添加新功能调用
try:
    print("\n开始高级分析...")

    # 1. LDA主题模型分析
    lda_model, topic_keywords, dictionary, corpus = analyze_topics(results_df, num_topics=5)

    # 输出主题关键词
    print("\n=== 主题关键词 ===")
    for topic_id, keywords in topic_keywords.items():
        print(f"主题 {topic_id}: {' '.join(keywords[:10])}")

    # 2. 时间序列趋势分析（假设数据包含publish_time列，需根据实际数据调整）
    # 注意：若数据中没有时间列，需先添加或跳过此步骤
    if 'publish_time' in merged_df.columns:
        monthly_proportion = analyze_time_series(results_df, time_column='publish_time')
    else:
        print("警告: 数据中缺少时间列，跳过时间序列分析")

    # 3. 情感-主题联合分析
    if 'lda_model' in locals():  # 确保LDA模型已创建
        topic_df = analyze_sentiment_topic(lda_model, results_df, dictionary, corpus)

except Exception as e:
    print(f"高级分析出错: {str(e)}")

print("\n完整分析完成!")


开始高级分析...

=== 开始主题模型分析 ===
高级分析出错: doc2bow expects an array of unicode tokens on input, not a single string

完整分析完成!
